### Approach-2
Here I flatten the pixels across columns and used xgboost to predict the non-zero coordinates.

In [71]:
# import required libraries
import numpy as np
from xgboost import XGBRegressor
from src.data.make_dataset import make_dataset
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error

In [72]:
# generate dataset
x_train, y_train, x_test, y_test, x_val, y_val = make_dataset(1000, 300, 300, [50,50,1])

In [73]:
# flatten the image pixel across columns
x_train_flatten = []
x_test_flatten = []
x_val_flatten = []

for i in range(len(x_train)) : 
    x_train_flatten.append(x_train[i].sum(axis = 0).reshape((50)))
    
for i in range(len(x_test)) : 
    x_test_flatten.append(x_test[i].sum(axis = 0).reshape((50)))
    x_val_flatten.append(x_val[i].sum(axis = 0).reshape((50)))

# convert into array
x_train_flatten = np.array(x_train_flatten)
x_test_flatten = np.array(x_test_flatten)
x_val_flatten = np.array(x_val_flatten)

In [74]:
# train multi-output regressor xgb
xgb = XGBRegressor(objective='reg:squarederror', max_depth=7, eta=0.2, subsample=0.8, colsample_bytree=0.8, seed=42)
multi_output_model = MultiOutputRegressor(xgb)
multi_output_model.fit(x_train_flatten, y_train)

# prediction on test set
predictions = multi_output_model.predict(x_test_flatten)

# combined predictions
print(predictions)

[[27.58505     5.27112   ]
 [26.27932    19.340902  ]
 [22.120174   45.732483  ]
 [29.104578   46.68063   ]
 [20.673643   44.68302   ]
 [26.960886   26.643595  ]
 [30.810713   11.3892975 ]
 [24.0322     37.754013  ]
 [27.5146     23.755907  ]
 [18.653847    0.40516773]
 [22.866566   39.63365   ]
 [27.208136   17.391651  ]
 [30.810713   11.3892975 ]
 [26.436264    4.2378874 ]
 [22.602518    9.289223  ]
 [29.104578   46.68063   ]
 [22.21176    33.741245  ]
 [24.903008   29.61981   ]
 [25.77625    43.650967  ]
 [22.21176    33.741245  ]
 [25.77625    43.650967  ]
 [25.807306   22.230124  ]
 [29.739523   21.345507  ]
 [29.104578   46.68063   ]
 [24.903008   29.61981   ]
 [22.286535   42.73398   ]
 [25.807306   22.230124  ]
 [27.933268    3.2922726 ]
 [22.019917   15.287255  ]
 [24.43363    14.31653   ]
 [23.44082    34.790543  ]
 [22.866566   39.63365   ]
 [22.866566   39.63365   ]
 [14.320544   12.440263  ]
 [24.43363    14.31653   ]
 [27.954618   10.277461  ]
 [22.286535   42.73398   ]
 

In [75]:
# Calculate mae for each co-ordinate
mae_x = mean_absolute_error(y_test[:, 0], predictions[:, 0])
mae_y = mean_absolute_error(y_test[:, 1], predictions[:, 1])

# Calculate average MAE (for multi-output)
avg_mae = (mae_x + mae_y) / 2

print('MAE X Coordinate: ', mae_x)
print('MAE Y Coordinate: ', mae_y)
print('Average MAE: ', avg_mae)

MAE X Coordinate:  12.576301
MAE Y Coordinate:  0.31677458
Average MAE:  6.446537494659424
